# 安装依赖

In [ ]:
!pip install omegaconf~=2.3.0
!pip install torch~=2.1.0
!pip install tqdm
!pip install einops
!pip install vector_quantize_pytorch
!pip install transformers~=4.41.1
!pip install vocos
!pip install IPython
!pip install numpy

# Fix seeds

In [ ]:
def fix(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
import torch
torch._dynamo.config.cache_size_limit = 64
torch._dynamo.config.suppress_errors = True
torch.set_float32_matmul_precision('high')
import ChatTTS
from IPython.display import Audio
torch.backends.cudnn.enabled = False
chat = ChatTTS.Chat()
chat.load_models()

import torch
rand_spk = chat.sample_random_speaker()

params_infer_code = {
  'spk_emb': rand_spk, # add sampled speaker 
  'temperature': .3, # using custom temperature
  'top_P': 0.7, # top P decode
  'top_K': 20, # top K decode
}
inputs_cn = """
chat T T S 是一款强大的对话式文本转语音模型。它有中英混读和多说话人的能力。
chat T T S 不仅能够生成自然流畅的语音，还能控制[laugh]笑声啊[laugh]，
停顿啊[uv_break]语气词啊等副语言现象[uv_break]。这个韵律超越了许多开源模型[uv_break]。
请注意，chat T T S 的使用应遵守法律和伦理准则，避免滥用的安全风险。[uv_break]'
""".replace('\n', '')

params_refine_text = {
  'prompt': '[oral_2][laugh_0][break_4]',
} 
audio_array_cn = chat.infer(inputs_cn, params_refine_text=params_refine_text, params_infer_code=params_infer_code,do_text_normalization=False)
torchaudio.save("output3.wav", torch.from_numpy(audio_array_en[0]), 24000)